## Task done by :- Pratham Uday Chandratre

### Problem Statement
### Create a machine learning model that can recommend movies to users based on their preferences

In [4]:
# import all important library 
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
# Load Dataset
mv = pd.read_csv("tmdb_5000_movies.csv")
cr = pd.read_csv("tmdb_5000_credits.csv")

In [7]:
# view movies dataset
mv.sample(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
1704,28000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.thebigshortmovie.com/,318846,"[{""id"": 974, ""name"": ""bank""}, {""id"": 4373, ""na...",en,The Big Short,The men who made millions from a global econom...,57.518472,"[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2015-12-11,133346506,130.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,This is a true story.,The Big Short,7.3,2607
4131,0,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 878, ""na...",NaN,335866,"[{""id"": 10349, ""name"": ""survival""}, {""id"": 131...",en,Circle,Could you trust a jury of your peers with your...,6.752141,"[{""name"": ""Votiv Films"", ""id"": 34455}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2015-10-02,0,87.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,In here… everything gets you killed.,Circle,6.0,333
3913,3000000,"[{""id"": 18, ""name"": ""Drama""}]",NaN,22013,"[{""id"": 178870, ""name"": ""evangelist""}, {""id"": ...",en,Elmer Gantry,An ex-football player becomes a popular evange...,6.734965,"[{""name"": ""United Artists"", ""id"": 60}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1960-06-19,10400000,146.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,If there was a dollar to be made - Gantry woul...,Elmer Gantry,7.3,41
119,150000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www2.warnerbros.com/batmanbegins/index....,272,"[{""id"": 486, ""name"": ""himalaya""}, {""id"": 779, ...",en,Batman Begins,"Driven by tragedy, billionaire Bruce Wayne ded...",115.040024,"[{""name"": ""DC Comics"", ""id"": 429}, {""name"": ""L...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2005-06-10,374218673,140.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Evil fears the knight.,Batman Begins,7.5,7359
4589,0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 9648, ""na...",NaN,43743,"[{""id"": 10183, ""name"": ""independent film""}]",en,Fabled,Joseph just broke up with his girlfriend and i...,0.003352,[],[],2002-01-01,0,84.0,[],Released,There once was a wolf named Lupold...,Fabled,0.0,0


In [8]:
# view credits Dataset
cr.sample(5)

,movie_id,title,cast,crew
4063,9517,Bully,"[{""cast_id"": 1, ""character"": ""Marty Puccio"", ""...","[{""credit_id"": ""52fe4503c3a36847f80b73ad"", ""de..."
3726,11826,Sexy Beast,"[{""cast_id"": 9, ""character"": ""Gary \u201eGal\u...","[{""credit_id"": ""52fe448d9251416c75038c2d"", ""de..."
1804,9092,Snow White: A Tale of Terror,"[{""cast_id"": 1, ""character"": ""Lady Claudia Hof...","[{""credit_id"": ""57792629c3a368145c000906"", ""de..."
3995,5900,The Dress,"[{""cast_id"": 2, ""character"": ""Van Tilt"", ""cred...","[{""credit_id"": ""52fe442bc3a36847f80864f3"", ""de..."
696,2275,The General's Daughter,"[{""cast_id"": 1, ""character"": ""Paul Brenner"", ""...","[{""credit_id"": ""56904a209251414570000083"", ""de..."


In [9]:
# check shape
mv.shape

(4803, 20)

In [10]:
# check shape
cr.shape

(4803, 4)

In [11]:
# marge Dataset  
df = mv.merge(cr,on="title")

In [12]:
# after merge check shape
df.shape

(4809, 23)

In [13]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [14]:

# drop unuse columns
# genres
# id
# keywords
# title
# overview
# cast
# crew

df= df[['genres','id','keywords','title','overview','cast','crew']]

In [15]:
# after drop
df.sample(5)

,genres,id,keywords,title,overview,cast,crew
4571,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",4107,"[{""id"": 7005, ""name"": ""northern ireland""}, {""i...",Bloody Sunday,The dramatised story of the Irish civil rights...,"[{""cast_id"": 3, ""character"": ""Ivan Cooper"", ""c...","[{""credit_id"": ""52fe43adc3a36847f8067123"", ""de..."
2835,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 36, ""name...",11202,"[{""id"": 1402, ""name"": ""general""}, {""id"": 1956,...",Patton,"""Patton"" tells the tale of General George S. P...","[{""cast_id"": 1, ""character"": ""Gen. George S. P...","[{""credit_id"": ""52fe440e9251416c75027827"", ""de..."
395,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",1581,"[{""id"": 65, ""name"": ""holiday""}, {""id"": 212, ""n...",The Holiday,"Two women, one (Cameron Diaz) from America and...","[{""cast_id"": 1, ""character"": ""Amanda Woods"", ""...","[{""credit_id"": ""52fe4301c3a36847f80332a7"", ""de..."
1823,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 28, ""n...",619,"[{""id"": 1156, ""name"": ""sister sister relations...",The Bodyguard,A former Secret Service agent grudgingly takes...,"[{""cast_id"": 17, ""character"": ""Rachel Marron"",...","[{""credit_id"": ""52fe425ec3a36847f8019133"", ""de..."
3794,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10769, ""...",20468,"[{""id"": 928, ""name"": ""hairdresser""}, {""id"": 19...",The Big Tease,Thinking he's competing in Los Angeles' hot Pl...,"[{""cast_id"": 1, ""character"": ""Crawford Mackenz...","[{""credit_id"": ""52fe43e8c3a368484e00519d"", ""de..."


In [16]:
# check null values in DataFrame
df.isna().sum()

genres      0
id          0
keywords    0
title       0
overview    3
cast        0
crew        0
dtype: int64

In [17]:
# Remove null values in Dataframe
df.dropna(inplace=True)

In [18]:
# check null values after Drop 
df.isna().sum()

genres      0
id          0
keywords    0
title       0
overview    0
cast        0
crew        0
dtype: int64

In [19]:
# check duplicate values in DataFrame
df.duplicated().sum()

0

In [20]:
# check 0 index values of genres in DataFrame
df.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [21]:
# create function for Extraction genres columns only name  
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [22]:
# apply function 
df['genres']=df['genres'].apply(convert)

In [23]:
# after Extraction Dataframe
df.sample(5)

,genres,id,keywords,title,overview,cast,crew
3831,[Comedy],9427,"[{""id"": 1252, ""name"": ""suicide attempt""}, {""id...",The Full Monty,"Sheffield, England. Gaz, a jobless steelworker...","[{""cast_id"": 1, ""character"": ""Gaz"", ""credit_id...","[{""credit_id"": ""52fe44f6c3a36847f80b45e9"", ""de..."
2545,"[Thriller, Drama]",134411,"[{""id"": 494, ""name"": ""father son relationship""...",Snitch,Construction company owner John Matthews learn...,"[{""cast_id"": 12, ""character"": ""John Matthews"",...","[{""credit_id"": ""52fe4bd0c3a368484e19a431"", ""de..."
1653,"[Drama, Mystery, Romance, Thriller]",11208,"[{""id"": 2038, ""name"": ""love of one's life""}, {...",Wicker Park,"Matthew, a young advertising executive in Chic...","[{""cast_id"": 1, ""character"": ""Matthew"", ""credi...","[{""credit_id"": ""52fe44119251416c75027b89"", ""de..."
5,"[Fantasy, Action, Adventure]",559,"[{""id"": 851, ""name"": ""dual identity""}, {""id"": ...",Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,"[{""cast_id"": 30, ""character"": ""Peter Parker / ...","[{""credit_id"": ""52fe4252c3a36847f80151a5"", ""de..."
2190,"[Action, Comedy, Crime, Fantasy]",9470,"[{""id"": 780, ""name"": ""kung fu""}, {""id"": 2343, ...",Kung Fu Hustle,"Set in Canton, China in the 1940s, the story r...","[{""cast_id"": 24, ""character"": ""Sing"", ""credit_...","[{""credit_id"": ""52fe44fbc3a36847f80b5b19"", ""de..."


In [24]:
# apply function
df['keywords']=df['keywords'].apply(convert)

In [25]:
# after Extraction Dataframe
df.sample(5)

,genres,id,keywords,title,overview,cast,crew
2210,"[Comedy, Drama]",6963,"[new york, chicago, midlife crisis, television...",The Weather Man,"A Chicago weather man, separated from his wife...","[{""cast_id"": 3, ""character"": ""David Spritz"", ""...","[{""credit_id"": ""52fe446bc3a36847f8094f8b"", ""de..."
375,"[Comedy, Science Fiction]",2277,"[android, hologram, freedom, futuristic, robot]",Bicentennial Man,"Richard Martin buys a gift, a new NDR-114 robo...","[{""cast_id"": 1, ""character"": ""Andrew Martin"", ...","[{""credit_id"": ""52fe4349c3a36847f8048a5f"", ""de..."
1345,"[Drama, Action, Thriller]",2163,"[california, bank, ransom, car breakdown, kidn...",Breakdown,When his SUV breaks down on a remote Southwest...,"[{""cast_id"": 15, ""character"": ""Jeffrey \"" Jeff...","[{""credit_id"": ""52fe433cc3a36847f8044b9d"", ""de..."
2759,"[Fantasy, Drama, Comedy]",492,"[individual, transvestism, sexual identity, wi...",Being John Malkovich,Spike Jonze’s debut feature film is a love sto...,"[{""cast_id"": 5, ""character"": ""Craig Schwartz"",...","[{""credit_id"": ""52fe4249c3a36847f80128f7"", ""de..."
3409,"[Romance, Western]",8879,"[gunslinger, showdown, marshal, blackmail, min...",Pale Rider,A small gold mining camp is terrorised by a ru...,"[{""cast_id"": 9, ""character"": ""Preacher"", ""cred...","[{""credit_id"": ""52fe44c3c3a36847f80a8a4f"", ""de..."


In [26]:
# check 0 index values of cast in DataFrame
df.iloc[0].cast

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [27]:
# create function for Extraction cast columns only name  
def convert_cast(obj):
    l=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter != 3:
            l.append(i['name'])
            counter +=1
        else :
            break
    return l

In [28]:
# apply function
df['cast']=df['cast'].apply(convert_cast)

In [29]:
# after Extraction Dataframe
df.sample(2)

,genres,id,keywords,title,overview,cast,crew
971,"[Comedy, Drama, Romance]",12220,"[love of one's life, therapist, psychology, we...",The Story of Us,Ben and Katie Jordan are a married couple who ...,"[Bruce Willis, Michelle Pfeiffer, Rob Reiner]","[{""credit_id"": ""52fe44cc9251416c750414eb"", ""de..."
3763,"[Action, Comedy, Romance]",47816,[sport],3 Strikes,Brian Hooks plays a character who is just rele...,"[Brian Hooks, N'Bushe Wright, Faizon Love]","[{""credit_id"": ""52fe4748c3a36847f812cee5"", ""de..."


In [30]:
# check 0 index values of crew in DataFrame
df.iloc[0].crew

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [31]:
# create function for Extraction cast columns only name 
def fatch(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job']== 'Director':
            l.append(i['name'])
            break
    return l

In [32]:
# apply function
df['crew']=df['crew'].apply(fatch)

In [33]:
# after Extraction Dataframe
df.head(2)

,genres,id,keywords,title,overview,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


In [34]:
# apply lambda function for split overview columns
df['overview']=df['overview'].apply(lambda x:x.split())

In [35]:
# after apply lambda function 
df.head()

,genres,id,keywords,title,overview,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...",John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [36]:
# Replace all space in all columns
df['genres']=df['genres'].apply(lambda x:[i.replace(" ","")for i in x])
df['keywords']=df['keywords'].apply(lambda x:[i.replace(" ","")for i in x])
df['overview']=df['overview'].apply(lambda x:[i.replace(" ","")for i in x])
df['cast']=df['cast'].apply(lambda x:[i.replace(" ","")for i in x])
df['crew']=df['crew'].apply(lambda x:[i.replace(" ","")for i in x])

In [37]:
# After replace white space
df.sample(5)

,genres,id,keywords,title,overview,cast,crew
1270,"[Action, Comedy, Thriller]",7288,[],Duplex,"[When, a, young, couple, buys, their, dream, h...","[BenStiller, DrewBarrymore, EileenEssell]",[DannyDeVito]
1826,"[Drama, Thriller]",162903,"[journalist, biography, internet, informationl...",The Fifth Estate,"[A, look, at, the, relationship, between, Wiki...","[BenedictCumberbatch, DanielBrühl, AnthonyMackie]",[BillCondon]
2347,"[War, Comedy, Drama]",10364,"[worldwarii, island, bomber, pianosa, american]",Catch-22,"[A, bombardier, in, World, War, II, tries, des...","[AlanArkin, MartinBalsam, RichardBenjamin]",[MikeNichols]
4398,"[Comedy, Crime]",16155,"[robbery, organizedcrime, gangster, safecracke...",Safe Men,"[Two, untalented, singers, are, mistaken, for,...","[SamRockwell, SteveZahn, PaulGiamatti]",[JohnHamburg]
1783,"[Action, Adventure, Comedy, Family]",14560,"[securityguard, duringcreditsstinger]",Paul Blart: Mall Cop,"[Mild-mannered, Paul, Blart, (Kevin, James), h...","[KevinJames, KeirO'Donnell, JaymaMays]",[SteveCarr]


In [39]:
# create new columns and marge columns
df['tags']= df['overview'] + df['genres'] + df['keywords'] + df['cast'] + df['crew']

In [40]:
df.sample(3)

,genres,id,keywords,title,overview,cast,crew,tags
4654,[Drama],78307,"[prison, drugdealer, singlemother, teenagepreg...",On the Outs,"[Follows, the, choices, made, by, three, young...","[JudyMarte, PaolaMendoza, FlacoNavaja]",[LoriSilverbush],"[Follows, the, choices, made, by, three, young..."
3311,"[Action, Crime, Thriller]",389425,[],Code of Honor,"[Colonel, Robert, Sikes, is, on, a, mission, t...","[StevenSeagal, CraigSheffer, LouisMandylor]",[MichaelWinnick],"[Colonel, Robert, Sikes, is, on, a, mission, t..."
2069,"[Drama, Thriller, Mystery]",1073,"[bomb, terrorist, fbi, professor, paranoia, co...",Arlington Road,"[Threats, from, sinister, foreign, nationals, ...","[JeffBridges, TimRobbins, JoanCusack]",[MarkPellington],"[Threats, from, sinister, foreign, nationals, ..."


In [41]:
# create new dataFrame with 3 columns
new_df = df[['id','title','tags']]

In [42]:
new_df

,id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [43]:
# # check 0 index values of tags in DataFrame
new_df['tags'][0]

['In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.',
 'Action',
 'Adventure',
 'Fantasy',
 'ScienceFiction',
 'cultureclash',
 'future',
 'spacewar',
 'spacecolony',
 'society',
 'spacetravel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alienplanet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'loveaffair',
 'antiwar',
 'powerrelations',
 'mindandsoul',
 '3d',
 'SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron']

In [44]:
# apply lambda function for join all values in tags 
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\Prath\AppData\Local\Temp\ipykernel_9824\3789622323.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [45]:
# after join  
new_df['tags'][2]

'A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE. Action Adventure Crime spy basedonnovel secretagent sequel mi6 britishsecretservice unitedkingdom DanielCraig ChristophWaltz LéaSeydoux SamMendes'

In [46]:
new_df.sample(5)

,id,title,tags
1881,4518,Elizabeth,The story of the ascension to the throne and t...
2219,2639,Deconstructing Harry,This film tells the story of a successful writ...
4799,182291,On The Downlow,Isaac and Angel are two young Latinos involved...
2035,277,Underworld,Vampires and werewolves have waged a nocturnal...
1154,10398,Double Jeopardy,A woman framed for her husband's murder suspec...


In [47]:
# # apply lambda function for lower case all words in tags
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

C:\Users\Prath\AppData\Local\Temp\ipykernel_9824\770283760.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [48]:
new_df.head(4)

,id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...


In [49]:
ps = PorterStemmer()

In [50]:
# apply stemmer because one word repeated
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [51]:
new_df['tags']=new_df['tags'].apply(stem)

C:\Users\Prath\AppData\Local\Temp\ipykernel_9824\3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [52]:
# conver all movie for vector form
cv = CountVectorizer(max_features=5000,stop_words="english")

In [53]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [54]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [55]:
cv.get_feature_names()

C:\Users\Prath\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['000',
 '007',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '17th',
 '18',
 '18th',
 '18thcenturi',
 '19',
 '1910',
 '1920',
 '1930',
 '1940',
 '1944',
 '1950',
 '1950s',
 '1960',
 '1960s',
 '1970',
 '1970s',
 '1971',
 '1974',
 '1976',
 '1980',
 '1985',
 '1990',
 '1999',
 '19th',
 '19thcenturi',
 '20',
 '200',
 '2003',
 '2009',
 '20th',
 '21st',
 '23',
 '24',
 '25',
 '30',
 '300',
 '3d',
 '40',
 '50',
 '500',
 '60',
 '70',
 '80',
 'aaron',
 'aaroneckhart',
 'abandon',
 'abduct',
 'abigailbreslin',
 'abil',
 'abl',
 'aboard',
 'abov',
 'abus',
 'academ',
 'academi',
 'accept',
 'access',
 'accid',
 'accident',
 'acclaim',
 'accompani',
 'accomplish',
 'account',
 'accus',
 'ace',
 'achiev',
 'acquaint',
 'act',
 'action',
 'actionhero',
 'activ',
 'activist',
 'activities',
 'actor',
 'actress',
 'actual',
 'ad',
 'adam',
 'adamsandl',
 'adamshankman',
 'adapt',
 'add',
 'addict',
 'adjust',
 'admir',
 'admit',
 'adolesc',
 'adopt',
 'ador',
 'adrienbrodi',
 'adult'

In [56]:
similarty = cosine_similarity(vector)

In [57]:
similarty.shape

(4806, 4806)

In [58]:
sorted(list(enumerate(similarty[0])),reverse=True,key=lambda x:x[1])[1:6]

[(1216, 0.28676966733820225),
 (2409, 0.26901379342448517),
 (3730, 0.2605130246476754),
 (507, 0.255608593705383),
 (539, 0.2503866978335957)]

In [59]:
# create Recommender function
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarty[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]] .title)
    return

In [60]:
# finally done Recommender
recommend('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


In [61]:
# finally done Recommender
recommend('Batman Begins')

The Dark Knight
Batman
Batman
The Dark Knight Rises
10th & Wolf


In [62]:
# finally done Recommender
recommend('The Dark Knight Rises')

The Dark Knight
Batman Returns
Batman
Batman Forever
Batman Begins


In [63]:
# finally done Recommender
recommend("April Fool's Day")

Deterrence
A History of Violence
A Cinderella Story
Boynton Beach Club
Larry Crowne


In [64]:
# finally done Recommender
recommend('Witless Protection')

Dream House
London
Reign Over Me
The Valley of Decision
Alvin and the Chipmunks: The Road Chip


In [65]:
new_df.sample(10)

,id,title,tags
3851,89540,The Other Conquest,the film is a drama about the aftermath of the...
1846,77877,The Lucky One,u.s. marin sergeant logan thibault return from...
2239,256962,Little Boy,an eight-year-old boy is will to do whatev it ...
438,6964,Something's Gotta Give,harri sanborn is an age music industri exec wi...
2344,10535,Spy Hard,the evil gen. rancor ha hi sight set on world ...
1853,769,GoodFellas,"the true stori of henri hill, a half-irish, ha..."
2183,2355,Reign Over Me,a man who lost hi famili in the septemb 11 att...
4660,24869,Ink,invis forc exert power over us in our sleep. a...
1984,20697,Ready to Rumble,two slacker wrestl fan are devast by the oust ...
1775,56288,Spy Kids: All the Time in the World,"eight year after the third film, the oss ha be..."


# Thank you